### Fig3a,b

In [ ]:
library(maftools)
library(ComplexHeatmap)
library(dplyr)
library(circlize)
library(gridExtra)
library(ggplot2)
library(tidyr)

In [ ]:
cnv <- read.csv("./OCEC_cn.csv")
cnv_unique <- cnv[!duplicated(cnv$gene_name), ]
rownames(cnv_unique) <- cnv_unique$gene_name
cnv_unique <- cnv_unique[ , -2:-1]

colnames(cnv_unique) <- substr(colnames(cnv_unique), 2, nchar(colnames(cnv_unique)))
cnv_unique[cnv_unique>3] <- "Amplification"
cnv_unique[cnv_unique==3] <- "Gain"
cnv_unique[cnv_unique==2] <- ""
cnv_unique[cnv_unique==1] <- "Heterozygous Loss"
cnv_unique[cnv_unique==0] <- "Homozygous Deletion"

key_genes_cnv <- cnv_unique[c(
                            "ARID1A","CCND1","CCNE1","CDK4","CDK6","RB1","E2F1","E2F3","EGFR","ERBB2","KRAS","BRAF",
                            "PTEN","INPP4B","PIK3R1","PIK3CA","STK11","APC","CTNNB1","MYC","MYCN","TP53","ATM"
                        ),]

cn_matrix <- as.matrix(key_genes_cnv)


In [ ]:
clinical_data <- read.csv("./bulkRNA_clini_info.csv")
clinical_data$Tumor_Type <- factor(clinical_data$Tumor_Type,levels=c("OC","EC","CC"))
clinical_data <- clinical_data[order(clinical_data$Tumor_Type), ]

In [ ]:
serous_clinical_data <- clinical_data[(clinical_data$Tumor_SubType=="Serous"),] ## or "Endometrioid"
common_samples <- intersect(serous_clinical_data$sampleName_modified, colnames(cn_matrix))

serous_clinical_subset <- serous_clinical_data[serous_clinical_data$sampleName_modified %in% common_samples, ]
cn_matrix_subset <- cn_matrix[, common_samples, drop = FALSE]
serous_clinical_subset <- serous_clinical_subset[order(serous_clinical_subset$Tumor_Type),]

non_empty_counts <- apply(cn_matrix_subset, 1, function(x) sum(x != ""))
sorted_rows <- names(sort(non_empty_counts, decreasing = TRUE))
cn_matrix_sorted <- cn_matrix_subset[sorted_rows, , drop = FALSE]

OC_cn_matrix <- cn_matrix_sorted[,1:147]
EC_cn_matrix <- cn_matrix_sorted[,148:ncol(cn_matrix_sorted)]

cn_types <- c("Amplification","Gain","Heterozygous Loss","Homozygous Deletion")
cn_col_list <- c("#de4431","#f2aa70","#a7b9d2","#4580b8")

cn_colors <- setNames(cn_col_list, cn_types)

cn_alter_fun_list <- list()
for (cn_type in cn_types) {
  cn_alter_fun_list[[cn_type]] <- local({
    cn_type_copy <- cn_type
    function(x, y, w, h) {
      grid.rect(x, y, w, h, gp = gpar(fill = cn_colors[cn_type_copy], col = NA))
    }
  })
}

cn_alter_fun_list$background <- function(x, y, w, h) {
  grid.rect(x, y, w, h, gp = gpar(fill = "#ebebeb", col = NA))
}

cn_priority <- setNames(seq_along(cn_types), cn_types)

OC_num_matrix <- apply(OC_cn_matrix, 2, function(x) {
  sapply(x, function(y) ifelse(y %in% names(cn_priority), cn_priority[y], 0))
})
OC_num_matrix[OC_num_matrix==0] <- 5
OC_sample_order <- do.call(order, as.data.frame(t(OC_num_matrix)))

EC_num_matrix <- apply(EC_cn_matrix, 2, function(x) {
  sapply(x, function(y) ifelse(y %in% names(cn_priority), cn_priority[y], 0))
})
EC_num_matrix[EC_num_matrix==0] <- 5
EC_sample_order <- do.call(order, as.data.frame(t(EC_num_matrix)))

EC_sample_order <- EC_sample_order + 147
sample_order <- c(OC_sample_order,EC_sample_order)

order_matrix <- cn_matrix_sorted[,sample_order]
serous_clinical_subset_ordered <- serous_clinical_subset[
  match(colnames(order_matrix), serous_clinical_subset$sampleName_modified),
]

In [ ]:
# clinical_data <- serous_clinical_subset_ordered[,c('cluster',"Tumor_Type","Tumor_SubType","Age","Recurrence_Status","purity","Tumor_Grade","Tumor_Stage","MSI_state","treatment","Response","CIN_score",'immune_state')]
clinical_data <- serous_clinical_subset_ordered[,c('cluster',"Tumor_Type","Tumor_SubType","Age","Recurrence_Status","purity","Tumor_Grade","Tumor_Stage","MSI_state","treatment","Response","CIN_score")]
clinical_data$Response <- trimws(clinical_data$Response)
clinical_data$Response[is.na(clinical_data$Response)] <- ""
row.names(clinical_data) <- clinical_data$Pathology_ID
clinical_data$Pathology_ID <- NULL

# clinical_data <- clinical_data[sort_id_1,]
clinical_data[is.na(clinical_data)] <- "Unknown"
clinical_data[clinical_data==""] <- "Unknown"

clinical_data$CIN_score[clinical_data$CIN_score == "Unknown"] <- NA
clinical_data$CIN_score <- as.numeric(clinical_data$CIN_score)


CIN_color_fun <- colorRamp2(
  breaks = c(min(clinical_data$CIN_score, na.rm = TRUE), max(clinical_data$CIN_score, na.rm = TRUE)),
  colors = c("#fee2d5", "#cc5500")
)


top_annotations <- HeatmapAnnotation(
  df = clinical_data,
  col = list(
    cluster = c("1"="#B1BCF4", "2"="#65A56D", "3"="#EAB768","4"="#F4E88C","5"="#e32636","6"="#ffdbe0", 'Unknown' = "#dcdcdc"),
    Tumor_Type = c("OC" = "#ab526a", "EC" = "#d68081", "CC"="#2978b5", 'Unknown' = "#dcdcdc"),
    Recurrence_Status = c("1" = "#785190", "0" = "#6f87bb", 'Unknown' = "#dcdcdc"),
    MSI_state = c("MSI-low" = "#62b58f", "MSI-high" = "#e7cc5e", 'Unknown' = "#dcdcdc"),
    CIN_score = CIN_color_fun,
    ),
  na_col = "#dcdcdc",
  annotation_name_side = "left",
  annotation_legend_param = list(
    cluster = list(title = "Cluster", title_gp = gpar(fontsize = 13)),
    Tumor_Type = list(title = "Tumor_Type", title_gp = gpar(fontsize = 13)),
    Recurrence_Status = list(title = "Recurrence_Status", title_gp = gpar(fontsize = 13)),
    MSI_state = list(title = "MSI_state", title_gp = gpar(fontsize = 13)),
    CIN_score = list(
      title = "CIN_score", 
      title_gp = gpar(fontsize = 13),
      color_bar = "continuous"
    )
  )
)

In [ ]:
cn_onco <- oncoPrint(order_matrix,
          alter_fun = cn_alter_fun_list,
          col = cn_colors,
          top_annotation = top_annotations,
          remove_empty_columns = FALSE,
          remove_empty_rows = FALSE,
          column_order = 1:ncol(order_matrix),
        #   column_title = "Samples",
          column_title = NULL,
          right_annotation = NULL,
          left_annotation = NULL,
          # row_title = "Genes",
          show_column_names = FALSE,
          row_names_side = "right",
          show_row_names = TRUE)


In [ ]:
options(stringsAsFactors = F)
laml = read.maf(maf = './combined_OCEC_maf.maf')

laml_mut <- getGeneSummary(laml)
genes <- c("ARID1A","CCND1","CCNE1","CDK4","CDK6","RB1","E2F1","E2F3","EGFR","ERBB2","KRAS","BRAF",
                   "PTEN","INPP4B","PIK3R1","PIK3CA","ATK3","STK11","APC","CTNNB1","MYC","TP53","ATM")

maf_data <- subsetMaf(maf = laml, genes = genes, isTCGA = FALSE)
all_samples <- unique(laml@data$Tumor_Sample_Barcode)
mutations <- maf_data@data[, c("Hugo_Symbol", "Tumor_Sample_Barcode","Variant_Classification")]
mutations$Hugo_Symbol <- as.character(mutations$Hugo_Symbol)
mutations$Tumor_Sample_Barcode <- as.character(mutations$Tumor_Sample_Barcode)
mutations$Variant_Classification <- as.character(mutations$Variant_Classification)
mut_order <- c('Multi_Hit','Missense_Mutation','Nonsense_Mutation','Frame_Shift_Del','Frame_Shift_Ins','In_Frame_Del','In_Frame_Ins','Splice_Site')
laml@data$Variant_Classification <- factor(laml@data$Variant_Classification, levels = mut_order)

mut_matrix <- matrix("", nrow = length(genes), ncol = length(all_samples),
                     dimnames = list(genes, all_samples))
for (i in 1:nrow(mutations)) {
    gene <- mutations$Hugo_Symbol[i]
    sample <- mutations$Tumor_Sample_Barcode[i]
    variant <- as.character(mutations$Variant_Classification[i])

    
    if (mut_matrix[gene, sample] == "") {
      mut_matrix[gene, sample] <- variant
    } else {
      mut_matrix[gene, sample] <- "Multi_Hit"
    }
}

non_empty_counts <- apply(mut_matrix, 1, function(x) sum(x != ""))
sorted_rows <- names(sort(non_empty_counts, decreasing = TRUE))
mut_matrix_sorted <- mut_matrix[sorted_rows, , drop = FALSE]

new_colnames <- sapply(strsplit(colnames(mut_matrix_sorted), "_"), `[`, 1)
colnames(mut_matrix_sorted) <- new_colnames
mut_matrix_sorted <- mut_matrix_sorted[,colnames(order_matrix)]

mutation_types <- c('Missense_Mutation','Splice_Site','Nonsense_Mutation','Frame_Shift_Del','Frame_Shift_Ins','In_Frame_Ins','In_Frame_Del','Multi_Hit')
mutation_types <- factor(mutation_types, level=mutation_types)
mut_col_list <- c("#db6566","#b797c6","#edaa5c","#FFD19C","#7dc0a7","#79e7f2","#2f7fc1","#0d5886")
mut_colors <- setNames(mut_col_list, mutation_types)

mut_alter_fun_list <- list()
for (mut_type in mutation_types) {
  mut_alter_fun_list[[mut_type]] <- local({
    mut_type_copy <- mut_type
    function(x, y, w, h) {
      grid.rect(x, y, w, h, gp = gpar(fill = mut_colors[mut_type_copy], col = NA))
    }
  })
}

mut_alter_fun_list$background <- function(x, y, w, h) {
  grid.rect(x, y, w, h, gp = gpar(fill = "#ebebeb", col = NA))
}


In [ ]:
mut_onco <- oncoPrint(mut_matrix_sorted,
          alter_fun = mut_alter_fun_list,
          col = mut_colors,
          top_annotation = NULL,
          remove_empty_columns = FALSE,
          remove_empty_rows = FALSE,
          column_order = 1:ncol(mut_matrix_sorted),
        #   column_title = "Samples",
          column_title = NULL,
          right_annotation = NULL,
          left_annotation = NULL,
          # row_title = "Genes",
          show_column_names = FALSE,
          row_names_side = "right",
          show_row_names = TRUE)


In [ ]:
draw(cn_onco %v%  mut_onco, heatmap_legend_side = "right")
